In [2]:
import numpy as np
import scipy as sp
import math
import functools as ft
from qiskit.quantum_info import Statevector, SparsePauliOp, Operator
from matplotlib import pyplot as plt

#### Constants

In [3]:
# Can't use m and g too big because too massive and too much coupling means the particles are not going to move. 
# It's not just their ratio that matters but also the numerical values because of the kinetic term

#m = 0.5
#g = math.sqrt(m*16/3)

g = 2.
m = 1.
nQ = 6

In [4]:
g

1.632993161855452

#### Helper Functions

In [7]:
# Define the Pauli Matrices

x = np.matrix([[0.,1.],[1.,0.]])
y = np.matrix([[0.,-1j],[1j,0.]])
z = np.matrix([[1.,0.],[0.,-1.]])
id = sp.sparse.identity(2)

proj0 = (z + id)/2.
proj1 = (id - z)/2.

In [8]:
def ry(angle):
    ang = angle/2
    return np.matrix([[math.cos(ang),-math.sin(ang)],[math.sin(ang),math.cos(ang)]])

In [9]:
# Pad operator on one qubit with identities to expand it to dimension of the full hilbert space
# "op" is the 1 qubit operator. For example Pauli x or Projection
# "site" is the site where the operator is to be placed
# "total" is the total dimension of the full hilbert space

def pad_op(site,total,op):
    return ft.reduce(sp.sparse.kron, [sp.sparse.identity(2**(site)),op,sp.sparse.identity(2**(total-site-1))])

In [ ]:
def overlap(state1,state2):
    return np.abs(state1.conj().T @ state2)[0]

# Construct Hamiltonian for Scipy calculations

In [10]:
# Calculate the electric energy at each site

def He_persite_scipy(nq,g,site):
    return 3 * g**2/8 * pad_op(site,nq,proj1)

In [ ]:
# Plot the electric energy at each site for all the sites

def plot_E_per_site(state,nq):      # state is a csr or csc array
    E_exps = []
    for loc in range(nq):
        E_exps += [(np.real(state.conj().T @ He_persite_scipy(nq,g,loc) @ state)).toarray()[0][0]] # Should be CT(state).H.state
    plt.plot(list(range(nq)), E_exps,'b.',linestyle = "-", label = 'Electric Hamiltonian Expectation Value')
    return E_exps

# Construct the Hamiltonian for Qiskit VQE

#### Helper Functions

In [38]:
# Generate Pauli operators
# "op" is the Pauli operator. For example Pauli x or Pauli z
# "loc" is the site where the operator is to be placed

def pauli_op(op,loc,nq,cons = 1):
    operator = SparsePauliOp.from_sparse_list([(op, [loc], cons)], num_qubits = nq)
    return operator

In [39]:
# Generate projection operators P0 or P1 with dimension of the full hilbert space. 
# "which" is 0 for P0 and 1 for P1. 
# "loc" is the site where the projection is to be placed.

def projection(which, loc, nq):
    if which == 0:
        operator = SparsePauliOp.from_sparse_list([("Z", [loc], 1), ("I", [loc], 1)], num_qubits=nq)/2
    elif which == 1:
        operator = SparsePauliOp.from_sparse_list([("Z", [loc], -1), ("I", [loc], 1)], num_qubits=nq)/2
    else:
        raise Exception("Type of projection operator not supported. Choose 0 or 1.")
    return operator

#### Kinetic Hamiltonian

In [40]:
def Hkin(nq):
    
    # Initialize "0" operators
    op1 = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)
    op2 = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)

    for i in range(nq):
        op1 += projection(0,i,nq).dot(pauli_op("X",(i+1)%nq,nq)).dot(projection(0,(i+2)%nq,nq))
        op2 += projection(1,i,nq).dot(pauli_op("X",(i+1)%nq,nq)).dot(projection(1,(i+2)%nq,nq))
        
    return (math.sqrt(2)*op1/2 + op2/math.sqrt(2)/2).simplify()

#### Mass Hamiltonian

In [41]:
def Hm(nq,m):
    op = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)

    for i in range(nq):
        op += projection(0,i,nq).dot(projection(1,(i+1)%nq,nq)) + projection(1,i,nq).dot(projection(0,(i+1)%nq,nq))
        
    return (m*op).simplify()

In [1]:
def Hm_persite(site, nq = nQ,m = m):
    return m*(projection(0,site,nq).dot(projection(1,(site+1)%nq,nq)) + projection(1,site,nq).dot(projection(0,(site+1)%nq,nq)))

NameError: name 'nQ' is not defined

#### Electric Hamiltonian

In [42]:
def He(nq,g):
    op = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)
    
    for i in range(nq):
        op += projection(1,i,nq) 
        
    return (3*g**2/8*op).simplify()

In [43]:
def He_persite(site,nq = nQ,g = g):
    return 3 * g**2/8 * projection(1,site,nq) 

#### Full Hamiltonian with mass m and gauge coupling g

In [44]:
def Hfull(nq,m,g):
    return (Hkin(nq)+He(nq,g)+Hm(nq,m)).simplify()

#### Result Handling etc..

In [ ]:
def plotAmpDict(AmpDict,isprob = False):
    if isprob == True:
        plt.plot(list(range(2**nQ)), np.sqrt(list(AmpDict.values())),label = 'QC Simulator')
    else: 
        plt.plot(list(range(2**nQ)), AmpDict.values(),label = 'QC Simulator')
    plt.xlabel("State", fontsize=16)
    plt.ylabel("Amplitude", fontsize=16)
    plt.legend()
    plt.show() 

In [ ]:
# This function handles results from the sampler by padding the locations with no values 0.
# Generates a list of probabilities in order of state 0-31
# Could also want amplitude here instead of the probability
# keys of the dictionary are the state numbers 0-2**nQ. Would generate all 0s if the keys are bit strings. 

def padDict(dicta):
    ordered_prob = []
    for i in range(2**nQ):
        if i in dicta.keys(): 
            ordered_prob.append(dicta[i])
        else: ordered_prob.append(0)
    return ordered_prob 

In [ ]:
def index_from_bitstring(bitstring):
    bitstring_reversed = reversed(bitstring)
    index = 0
    for i,c in enumerate(bitstring_reversed):
        if c == '1':
            index += np.power(2, i)
    return index

# works when keys are bitstrings
def padbitstringDict(dictaa,nq = nQ):
    ordered_prob = [0]*2**nq
    for item in dictaa.keys():
        ordered_prob[index_from_bitstring(item)] = dictaa[item]
    return ordered_prob 